# Wrapping a Model for Serving in Seldon

Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   
 * If you are viewing this on the web then to run this notebook install jupyter and follow the steps below and click on wrap_model.ipynb when the jupyter page is shown:
 ```
 git clone https://github.com/SeldonIO/seldon-core-launcher.git
 cd seldon-core-launcher/seldon-core/getting_started/wrap-model
 jupyter notebook
 ```

## Requirements


 * You have a running cluster installed via the Google Marketplace with all the defaults including:
    * NodePort for the Seldon API OAuth Gateway. This gateway is used to connect your business apps to your running models via REST and gRPC.
    * The cluster is running in the default namespace
 
## Dependencies

You will need install the following dependencies:

 * [S2I](https://github.com/openshift/source-to-image)
 * [sklearn](http://scikit-learn.org/stable/) to train the model
 * [grpcio-tools](https://grpc.io/docs/quickstart/python.html) to allow testing using gRPC
 
Sklearn and gRPC packages can easily be installed using pip:
```bash
pip install sklearn
pip install grpcio-tools
``` 

# Train a model
We will train a model for the Iris classification task. In this simple dataset we try to classifier a type if Iris based on its petal and sepal length. 
<img src="./iris.jpg" alt="iris" title="iris"/>

In [1]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn import datasets

def main():
    clf = LogisticRegression()
    p = Pipeline([('clf', clf)])
    print('Training model...')
    p.fit(X, y)
    print('Model trained!')

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(p, filename_p)
    print('Model saved!')
    
if __name__ == "__main__":
    print('Loading iris data set...')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print('Dataset loaded!')
    main()


Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in IrisClassifier.sav
Model saved!


# Wrap runtime code
We will now create runtime code to get predictions and wrap this into a Docker container so it can be launched inside seldon-core. The runtime code is defined in the file IrisClassifier.py and is show below:

In [2]:
!pygmentize IrisClassifier.py

from sklearn.externals import joblib

class IrisClassifier(object):

    def __init__(self):
        self.model = joblib.load('IrisClassifier.sav')
        self.class_names = ["iris-setosa","iris-vericolor","iris-virginica"];

    def predict(self,X,features_names):
        return self.model.predict_proba(X)


To wrap this model we will use S2I (Source to Image) which will take this code and wrap it with a REST server so it can be deployed inside seldon. The wrapping process needs some details on where the runtime code is and what type of endpoint to create: REST or gRPC. This is held in the .s2i folder in a file called *environment*. This file is shown below:

In [3]:
!cat .s2i/environment

MODEL_NAME=IrisClassifier
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0


We are now ready to wrap the model using s2i. We need to choose a Seldon builder image. Seldon provides builder images for python (2 and 3) and also R and Java. In this case we choose python 3. We also need to provide the name of the image we wish to build. Replace *my-repo* with the name of your Docker repository. We also provide a requirements.txt with the needed python packages. To get more details on wrapping python models for seldon-core see [here](https://github.com/SeldonIO/seldon-core/blob/master/docs/wrappers/python.md).

**change below to your Docker repository**

In [4]:
%env DOCKER_REPO=seldonio

env: DOCKER_REPO=seldonio


In [5]:
!s2i build . seldonio/seldon-core-s2i-python3 ${DOCKER_REPO}/sklearn-iris:0.1

---> Installing application source...
---> Installing dependencies ...
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


Let's check the image has been built.

In [6]:
!docker images | grep sklearn-iris

seldonio/sklearn-iris                                          0.1                 c65c10203966        2 seconds ago       1.25GB
seldonio/sklearn-iris                                          <none>              e3e2ac61aa79        5 hours ago         1.25GB
seldonio/sklearn-iris                                          <none>              ed98829dd467        6 hours ago         1.25GB
seldonio/sklearn-iris                                          <none>              8c83a438f912        7 hours ago         1.25GB


Now push this image to your repo so we can use it inside your seldon-core cluster

In [7]:
!docker push ${DOCKER_REPO}/sklearn-iris:0.1

The push refers to repository [docker.io/seldonio/sklearn-iris]

4e067887: Preparing 
cd490e05: Preparing 
ed9127c2: Preparing 
c90ed723: Preparing 
69d7992c: Preparing 
e783060b: Preparing 
4c18b186: Preparing 
4e44553f: Preparing 
50bdb983: Preparing 
5c484bde: Preparing 
34df1f1a: Preparing 
8bf2f209: Preparing 
b36b7599: Preparing 
d5dcea45: Preparing 
d020c11b: Preparing 
4afd042f: Preparing 
e067887: Pushed   269.4MB/267.3MB14A0.1: digest: sha256:19d38e42c6bcd71bf91e9a7c38c93e3c2aa067ba0089be3b36c7d99202c43e1f size: 3899


# Deploy model 
We will now deploy a runtime graph to serve our model on our seldon-core cluster

## Prerequistes

 * You have a running cluster installed via the Google Marketplace with all the defaults including:
    * NodePort for the Seldon API OAuth Gateway. This gateway is used to connect your business apps to your running models via REST and gRPC.
    * The cluster is running in the default namespace
 
 You will need to install some software for this demo:
 
 
 * Install the [requests library](http://docs.python-requests.org/en/master/) to allow you to make REST calls to the Seldon API gateway.
 * Install [python grpc tools](https://grpc.io/docs/quickstart/python.html) to allow you to make gRPC calls to the Seldon API gateway.
 * Install [graphviz](https://pypi.org/project/graphviz/) a package to display graphs.


## Set up REST and gRPC methods

**Ensure you port forward the seldon api-server REST and GRPC ports**, do this in separate terminals:

REST:
```
kubectl port-forward $(kubectl get pods -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') 8002:8080
```

GRPC:
```
kubectl port-forward $(kubectl get pods -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') 8003:5000
```

In [8]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
try:
    from commands import getoutput # python 2
except ImportError:
    from subprocess import getoutput # python 3

API_HTTP="localhost:8002"
API_GRPC="localhost:8003"

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+API_HTTP+"/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    print(response.text)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["sepallengthcm","sepalwidthcm","petallengthcm","petalwidthcm"],"tensor":{"shape":[1,4],"values":[5.1,3.5,1.4,0.2]}}}
    response = requests.post(
                "http://"+API_HTTP+"/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["sepallengthcm","sepalwidthcm","petallengthcm","petalwidthcm"],
            tensor = prediction_pb2.Tensor(
                shape = [1,4],
                values = [5,1,3.5,1.4,0.2]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(API_GRPC)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)


We need to describe a runtime graph for our iris model so we can deploy it using seldon-core. This is shown below. We will update ${DOCKER_REPO} with our given docker repository we set above.

In [9]:
!pygmentize TMPL_deployment.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "sklearn-iris-example"
    },
    "spec": {
        "name": "sklearn-iris-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "${DOCKER_REPO}/sklearn-iris:0.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "sklearn-iris-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                

Let's update our deployment.json template with the name of our Docker Repo and apply this on our cluster.

In [10]:
!cat TMPL_deployment.json | envsubst | kubectl apply -f - 

seldondeployment "sklearn-iris-example" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1 for all components**.

In [11]:
!kubectl get seldondeployments sklearn-iris-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:sklearn-iris-deployment-classifier-svc-orch replicas:1 replicasAvailable:1] map[name:sklearn-iris-deployment-classifier-sklearn-iris-classifier-0 replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request
We will get an OAuth token using the key and secret we specified in the graph above and then call the REST endpoint of the API gateway with some random data.

In [12]:
rest_request()

{"access_token":"93f81a9e-4658-4e34-8f0a-cf232c125a20","token_type":"bearer","expires_in":42935,"scope":"read write"}
{
  "meta": {
    "puid": "g15i4319ihek4hf89tphs5dspu",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["iris-setosa", "iris-vericolor", "iris-virginica"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.8796816489561845, 0.12030753790658998, 1.0813137225507727E-5]
    }
  }
}


## Tear Down

In [13]:
!cat TMPL_deployment.json | envsubst | kubectl delete -f -

seldondeployment "sklearn-iris-example" deleted


# Next Steps

There is extensive documentation on using seldon-core at https://github.com/SeldonIO/seldon-core